This script collect methods for finance data pipeline

Workflow: collect finance data by company symbol -> preprocess data to get time, article for each symbol -> save into dataframe file
    

# method 1
from github: jalajthanaki/stock_price_prediction
<br><br>
06282020: error, can not find newsapi.articles


In [4]:
"""
About:
Python wrapper for the New York Times Archive API
https://developer.nytimes.com/article_search_v2.json
"""
from newsapi.articles import Articles
import sys,json
import requests

key = '522497f7b4b940b7946eeed6909ed817'
params = {}
api = Articles(API_KEY=key)
reload(sys)
sys.setdefaultencoding('utf8')


class APIKeyException(Exception):
    def __init__(self, message): self.message = message


class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message


class ArchiveAPI(object):
    def __init__(self, key=None):
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise APIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None, ):
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1851
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        #print url ##boting: remove error message
        r = requests.get(url)
        return r.json()




ModuleNotFoundError: No module named 'newsapi.articles'

In [5]:
import newsapi 

In [8]:
from newsapi.articles import Articles

ModuleNotFoundError: No module named 'newsapi.articles'

In [7]:
dir(newsapi)

['NewsApiClient',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'const',
 'newsapi_auth',
 'newsapi_client',
 'newsapi_exception',
 'utils']

In [ ]:
years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
api = ArchiveAPI('522497f7b4b940b7946eeed6909ed817')
for year in years:
    for month in months:
        mydict = api.query(year, month)
        file_str = './data/nytimes' + str(year) + '-' + '{:02}'.format(
            month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

# method 2
from github: jasonyip184/StockSentimentTrading
 
business times

In [14]:
import nltk
nltk.download('vader_lexicon')

import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /Users/wangbo-
[nltk_data]     ting/nltk_data...


In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

29 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ad-boycott-campaign-to-go-global-organisers-say
27 Jun 2020 https://www.businesstimes.com.sg/technology/unilever-to-stop-advertising-on-facebook-twitter-and-instagram-in-us
27 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ramps-up-efforts-to-curb-hateful-content-in-ads
26 Jun 2020 https://www.businesstimes.com.sg/technology/verizon-suspends-advertising-on-facebook-joins-growing-boycott
24 Jun 2020 https://www.businesstimes.com.sg/consumer/ben-jerrys-joins-facebook-ad-boycott-over-hate-speech
23 Jun 2020 https://www.businesstimes.com.sg/government-economy/biden-campaign-asks-twitter-facebook-to-remove-trump-posts-bashing-mail-in-voting
23 Jun 2020 https://www.businesstimes.com.sg/technology/microsoft-ends-game-streaming-teams-up-with-facebook
19 Jun 2020 https://www.businesstimes.com.sg/government-economy/facebook-pulls-trump-ads-over-nazi-symbol
18 Jun 2020 https://www.businesstimes.com.sg/technology/faceb

13 Feb 2020 https://www.businesstimes.com.sg/technology/facebook-owned-whatsapp-says-it-has-2-billion-users
08 Feb 2020 https://www.businesstimes.com.sg/technology/twitter-says-facebook-messenger-accounts-hacked
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-to-remove-virus-misinformation-after-who-declares-global-emergency
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebooks-revenue-rises-again-but-more-slowly-than-ever
30 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-results-beat-forecasts-but-shares-take-a-hit
29 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-rolls-out-tool-globally-to-clear-third-party-data
28 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-asks-employees-to-put-off-non-essential-travel-to-china
23 Jan 2020 https://www.businesstimes.com.sg/banking-finance/facebook-spurred-central-banks-to-study-digital-currencies-ex-boj-exec
22 Jan 2020 https://www.businesstimes.com.sg/banking-finance/vodaf

In [17]:
date_sentiment

18 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-must-disclose-app-records-for-probe-us-judge-rules
 Hello, your email is unverified. Please confirm  
0.9844
10 Jan 2020 https://www.businesstimes.com.sg/technology/lawmakers-facebooks-steps-to-tackle-deepfake-videos-inadequate
 Hello, your email is unverified. Please confirm  
-0.8632
08 Jan 2020 https://www.businesstimes.com.sg/consumer/facebooks-new-singapore-team-to-serve-china-advertising-business
 Hello, your email is unverified. Please confirm  
0.995
08 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-tightening-policy-against-deepfakes
 Hello, your email is unverified. Please confirm  
0.9126
07 Jan 2020 https://www.businesstimes.com.sg/government-economy/trump-says-zuckerberg-told-him-hes-facebooks-number-one
 Hello, your email is unverified. Please confirm  
-0.628
28 Dec 2019 https://www.businesstimes.com.sg/banking-finance/facebooks-libra-has-failed-in-current-form-says-swiss-president
 Hello, yo

In [18]:
date_sentiment

NameError: name 'date_sentiment' is not defined

# method 3
from github dineshdaultani/StockPredictions

NYTimes

need an api 'key'

In [22]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 00:10:29 2017
@author: Dinesh
"""
# Below part is not required
########## News API ########################################################
# from newsapi import NewsAPI

# key = '96af62a035db45bda517a9ca62a25ac3'
# params = {}
# api = NewsAPI(key)
# sources = api.sources(params)
# articles = api.articles(sources[0]['id'], params)

################ NY Times API #############################################
import sys, csv, json
#reload(sys) #boting: reload has been removed since python 3.4
#sys.setdefaultencoding('utf8') #boting: sys.setdefaultencoding is not recommended for use


import requests
"""
About:
Python wrapper for the New York Times Archive API 
https://developer.nytimes.com/article_search_v2.json
"""

class APIKeyException(Exception):
    def __init__(self, message): self.message = message 

class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message 

class ArchiveAPI(object):
    def __init__(self, key=None):
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()


        

    

In [26]:
import os

In [27]:
# Replace below key with your NYTimes Developer key
api = ArchiveAPI('key')

#years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
years = [2016]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for year in years:
    for month in months:
        mydict = api.query(year, month)
        #file_str = '/Users/Dinesh/Documents/Project Stock predictions/data/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        if not os.path.exists('./news/nytimes/'):
            os.makedirs('./news/nytimes/')
        file_str = './news/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

# method 4
from github: WayneDW/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction

# method 5
github: Keess324/News_predict_Stock_Price_trend


In [10]:
# Import Library
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
#from datetime import datetime
from pandas_datareader import data as web
import os
import string
import requests
import nltk
from nltk import word_tokenize
from nltk.corpus import PlaintextCorpusReader

In [11]:
# Stock Price
def stock(stock_type, start, end):
    sp500stock = web.get_data_yahoo([stock_type], start,end)
    return sp500stock
    
stock_type = '^GSPC'
start=datetime.datetime(2016, 9, 26)
end=datetime.datetime(2018,10, 1)

stock_data = stock(stock_type, start, end)

In [12]:
stock_data.head(10)


Attributes,Adj Close,Close,High,Low,Open,Volume
Symbols,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,,
2016-09-26,2146.100098,2146.100098,2158.540039,2145.040039,2158.540039,3216170000
2016-09-27,2159.929932,2159.929932,2161.129883,2141.550049,2146.040039,3437770000
2016-09-28,2171.370117,2171.370117,2172.399902,2151.790039,2161.850098,3891460000
2016-09-29,2151.129883,2151.129883,2172.669922,2145.199951,2168.899902,4249220000
2016-09-30,2168.270020,2168.270020,2175.300049,2156.510010,2156.510010,4173340000
2016-10-03,2161.199951,2161.199951,2164.409912,2154.770020,2164.330078,3137550000
2016-10-04,2150.489990,2150.489990,2165.459961,2144.010010,2163.370117,3750890000
2016-10-05,2159.729980,2159.729980,2163.949951,2155.149902,2155.149902,3906550000


In [13]:
open_date = stock_data.index
len(open_date)

508

In [14]:
#Scraping news from the the corresponding date we found
date_loop_list = []
for i in range(len(open_date)):
    d1 = open_date[i]
    date_loop_list.append(d1.date())
print(len(date_loop_list))


508


In [22]:
#date_loop_list
date_loop_list[0]

datetime.date(2016, 9, 26)

In [ ]:
#if error message when running BeautifulSoup(page.content,'lxml'):
#FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?
#you need to install lxml in the command line
#pip3 install lxml

#after installing it, it still does not work,
#but BeautifulSoup(page.content,"html.parser")


In [33]:
date = date_loop_list[0]
url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
page = requests.get(url)    
#result = BeautifulSoup(page.content,"lxml")


In [36]:
result = BeautifulSoup(page.content,"html.parser")

In [37]:
result


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<style>@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-Book.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-Book.woff) format("woff");font-weight:400;font-style:normal;font-display:swap}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-BookItalic.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-BookItalic.woff) format("woff");font-weight:400;font-style:italic;font-display:optional}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-Light.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-Light.woff) format("woff");font-weight:300;font-style:normal;font-display:swap}@font-face{font-family:Retina;src:url(https://www.wsj.com/fonts/woffs/retina/Retina-LightItalic.woff2) format("woff2"),url(https://www.wsj.com/fonts/woffs/retina/Retina-LightItalic.woff) fo

In [38]:
# Web Scraping from Wall Street Journal
def news(dateloop):
#     date_list = [base - datetime.timedelta(days=x) for x in range(daterange)]
#     date_loop_list = []
#     for i in date_list:
#         dt = datetime.date.strftime(i, '%Y-%m-%d')
#         date_loop_list.append(dt)
    
    All_News=[]
    for date in dateloop:
        url = 'http://www.wsj.com/public/page/archive-'+ str(date) + '.html'
        page = requests.get(url)    
        result = BeautifulSoup(page.content,'lxml')
        UL = result.find_all('ul', class_ = 'newsItem')
        body1 = UL[0].find_all('p')
        everyday_results = []
        Headlines_Day = []
        Abstracts_Day = []
        for news_ in range(len(body1)):
            body2 = body1[news_].get_text()
            body3 = body2.strip().split('\n ')
            Headlines_item = body3[0]
            Abstracts_item = body3[-1]
            Headlines_Day.append(Headlines_item)
            Abstracts_Day.append(Abstracts_item)
        everyday_results = [date, Headlines_Day, Abstracts_Day]
        All_News.append(everyday_results)
    return All_News

In [39]:
output_news = news(date_loop_list)
# output_news

IndexError: list index out of range

In [17]:
def news_to_dataframe(output_news):
    news_df = pd.DataFrame()
    date = []
    news_title = []
    news_abstract = []
    for day in output_news:
        date.append(day[0])  # date
        news_title.append(day[1]) # news_title
        news_abstract.append(day[2]) # news_abstract
    news_df['Date'] = date
    news_df['News_title'] = news_title
    news_df['News_abstract'] = news_abstract
    
    return news_df


In [18]:
news_data = news_to_dataframe(output_news) 
news_data.head(10)

NameError: name 'output_news' is not defined

# method 6
github: WayneDW/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction

seems can not download ./input/tickerList.csv

In [1]:
#!/usr/bin/env python3
"""
Download the ticker list from NASDAQ and save as csv.
Output filename: ./input/tickerList.csv
"""
import csv
import sys

from urllib.request import urlopen

import numpy as np


def get_tickers(percent):
    """Keep the top percent market-cap companies."""
    assert isinstance(percent, int)

    file = open('./input/tickerList.csv', 'w')
    writer = csv.writer(file, delimiter=',')
    cap_stat, output = np.array([]), []
    for exchange in ["NASDAQ", "NYSE", "AMEX"]:
        url = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange="
        repeat_times = 10 # repeat downloading in case of http error
        for _ in range(repeat_times):
            try:
                print("Downloading tickers from {}...".format(exchange))
                response = urlopen(url + exchange + '&render=download')
                print('finish urlopen')
                content = response.read().decode('utf-8').split('\n')
                print('finish content')
                for num, line in enumerate(content):
                    print(num) #boting
                    line = line.strip().strip('"').split('","')
                    if num == 0 or len(line) != 9:
                        continue # filter unmatched format
                    # ticker, name, last_sale, market_cap, IPO_year, sector, industry
                    ticker, name, _, market_cap, _, _, _ = line[0:4] + line[5:8]
                    cap_stat = np.append(cap_stat, float(market_cap))
                    output.append([ticker, name.replace(',', '').replace('.', ''),
                                   exchange, market_cap])
                break
            except:
                continue

    for data in output:
        market_cap = float(data[3])
        if market_cap < np.percentile(cap_stat, 100 - percent):
            continue
        writer.writerow(data)


def main():
    if len(sys.argv) < 2:
        print('Usage: ./all_tickers.py <int_percent>')
        return
    top_n = sys.argv[1]
    get_tickers(int(top_n)) # keep the top N% market-cap companies


#if __name__ == "__main__":
#    main()

In [ ]:
import os
#set current directory to test the code
os.chdir('/Users/wangbo-ting/git/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/')
top_n = 20
get_tickers(int(top_n))

In [3]:
import os

isFile = os.path.isfile('../Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction/input/tickerList.csv')  
print(isFile) 

True


In [16]:
isDir = os.path.isdir('../Sentiment-Analysis-in-Event-Driven-Stock-Price-Movement-Prediction')  
print(isDir) 

True


In [9]:
os. getcwd()

'/Users/wangbo-ting/git/sentiment_analysis_stock'

In [3]:
url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download' 

In [ ]:
response = urlopen(url)

In [ ]:
urllib

# method 7
use free api

Finnhub.io 

only 2020 news available

if buy their api, you can get 20 years data

In [2]:
import requests

#use following process to extract company info from finnhub.io
#A. company = “SK.PA” / api_key = keys [‘finnhub’]
#B. url = “https://finnhub.io/api/v1/stock/profile?symbol={}&token={}".format(company, api_key)
#C. req = requests.get(url)

r = requests.get('https://finnhub.io/api/v1/company-news?symbol=AAPL&from=2020-04-30&to=2020-05-01&token=sign up for key')
#print(r.json())

In [17]:
r.json()[190].keys()

dict_keys(['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url'])

In [22]:
#show json structure
import json

JSONContent = r.json()
content = json.dumps(JSONContent, indent = 4, sort_keys=True)
print(content)

[
    {
        "category": "company",
        "datetime": 1588377600,
        "headline": "WhatsApp Suddenly Gets Powerful New Security Boost: Here\u2019s Why It Affects You",
        "id": 691320,
        "image": "https://thumbor.forbes.com/thumbor/fit-in/1200x0/filters%3Aformat%28jpg%29/https%3A%2F%2Fspecials-images.forbesimg.com%2Fimageserve%2F1203670407%2F0x0.jpg%3FcropX1%3D0%26cropX2%3D5000%26cropY1%3D259%26cropY2%3D3072",
        "related": "AAPL",
        "source": "https://www.forbes.com",
        "summary": "WhatsApp has been seriously boosted this week\u2014and from two very unlikely sources.",
        "url": "https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/"
    },
    {
        "category": "company",
        "datetime": 1588377600,
        "headline": "2020 iPhone Alert: Apple\u2019s New Price Changes Revealed",
        "id": 691319,
        "image": "https://thumbor.forbes.com/thumbor/fit-in

In [23]:
#assume keys in each json data are the same
QQQdf = pd.DataFrame(r.json())

In [25]:
QQQdf.shape

(200, 9)

In [127]:
QQQdf.head(20)

,category,datetime,headline,id,image,related,source,summary,url
0,company,1588377600,WhatsApp Suddenly Gets Powerful New Security B...,691320,https://thumbor.forbes.com/thumbor/fit-in/1200...,AAPL,https://www.forbes.com,WhatsApp has been seriously boosted this week—...,https://www.forbes.com/sites/zakdoffman/2020/0...
1,company,1588377600,2020 iPhone Alert: Apple’s New Price Changes R...,691319,https://thumbor.forbes.com/thumbor/fit-in/1200...,AAPL,https://www.forbes.com,Apple's most exciting iPhone 12 change is a sh...,https://www.forbes.com/sites/gordonkelly/2020/...
2,company,1588377600,Forget Google—Huawei Plans A Killer New Update...,691318,https://thumbor.forbes.com/thumbor/fit-in/1200...,AAPL,https://www.forbes.com,Is this enough to make you switch away from Go...,https://www.forbes.com/sites/zakdoffman/2020/0...
3,company,1588377600,Apple Watch Series 6 Leak Details 5 Great Inno...,687591,https://thumbor.forbes.com/thumbor/fit-in/1200...,AAPL,https://www.forbes.com,"A leak, confirmed by one of the hottest tipste...",https://www.forbes.com/sites/davidphelan/2020/...
4,company,1588377600,Market Slides More Than 2% to Begin May,666265,https://i-invdn-com.akamaized.net/redesign/ima...,AAPL,investing,Stocks Analysis by Zacks Investment Research c...,https://www.investing.com/analysis/market-slid...
5,company,1588377600,Buffett says coronavirus cannot stop America,25673,https://i-invdn-com.akamaized.net/trkd-images/...,AAPL,investing,Buffett says coronavirus cannot stop America,https://www.investing.com/news/coronavirus/buf...
6,company,1588377600,Market Slides More Than 2% to Begin May,25557,https://i-invdn-com.akamaized.net/redesign/ima...,AAPL,investing,Stocks Analysis by Zacks Investment Research c...,https://www.investing.com/analysis/market-slid...
7,company,1588377600,Tim Cook On Warren Buffett: 'I Don't Think The...,25473,https://cdn.benzinga.com/files/imagecache/og_i...,AAPL,benzinga,This article was originally published on May 6...,https://www.benzinga.com/news/19/05/13672277/t...
8,company,1588377600,Tim Cook On Warren Buffett: 'I Don't Think The...,25377,https://cdn.benzinga.com/files/imagecache/og_i...,AAPL,benzinga,This article was originally published on May 6...,https://www.benzinga.com/news/20/05/13672277/t...
9,company,1588377600,Point/Counterpoint: Zoom Bulls Vs. Bears,24435,https://i-invdn-com.akamaized.net/news/Compute...,AAPL,investing,Point/Counterpoint: Zoom Bulls Vs. Bears,https://www.investing.com/news/stock-market-ne...


In [31]:
QQQdf['headline'][0]

'WhatsApp Suddenly Gets Powerful New Security Boost: Here’s Why It Affects You'

In [29]:
QQQdf['summary'][0]

'WhatsApp has been seriously boosted this week—and from two very unlikely sources.'

In [33]:
url = QQQdf['url'][0]

In [34]:
#import pandas as pd

#df = pd.read_json(r.json() )

In [66]:
news_content = requests.get(url)

In [67]:
dir(news_content)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [83]:
news_content.content

b'<!DOCTYPE html><html lang="en"><head><title>WhatsApp Suddenly Gets Powerful New Security Boost: Here\xe2\x80\x99s Why It Affects You</title><meta charset="utf-8"><meta http-equiv="Content-Language" content="en_US"><link rel="shortcut icon" href="https://i.forbesimg.com/48X48-F.png"><meta name="referrer" content="no-referrer-when-downgrade"><link rel="canonical" itemprop="url" href="https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/"><link rel="amphtml" href="https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/amp/"><link rel="alternate" type="application/rss+xml" title="WhatsApp Suddenly Gets Powerful New Security Boost: Here\xe2\x80\x99s Why It Affects You - RSS" href="https://www.forbes.com/sites/zakdoffman/feed/"><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=5,minimum-scale=1,user-scalable=yes"

In [84]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(news_content.content, 'html.parser')

In [90]:
soup

<!DOCTYPE html>
<html lang="en"><head><title>WhatsApp Suddenly Gets Powerful New Security Boost: Here’s Why It Affects You</title><meta charset="utf-8"/><meta content="en_US" http-equiv="Content-Language"/><link href="https://i.forbesimg.com/48X48-F.png" rel="shortcut icon"/><meta content="no-referrer-when-downgrade" name="referrer"/><link href="https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/" itemprop="url" rel="canonical"/><link href="https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/amp/" rel="amphtml"/><link href="https://www.forbes.com/sites/zakdoffman/feed/" rel="alternate" title="WhatsApp Suddenly Gets Powerful New Security Boost: Here’s Why It Affects You - RSS" type="application/rss+xml"/><meta content="width=device-width,initial-scale=1,maximum-scale=5,minimum-scale=1,user-scalable=yes" name="viewport"/><meta content=

In [121]:
soup.figure.contents

['\n', <div>
 <img alt="In this photo illustration a WhatsApp messenger app logo..." data-height="3334" data-width="5000" src="https://specials-images.forbesimg.com/imageserve/1203670407/960x0.jpg?fit=scale"/>
 </div>, '\n', <figcaption>
 <p class="color-body light-text"><small>SOPA Images/LightRocket via Getty Images</small></p>
 </figcaption>, '\n']

In [98]:
pTag = soup.p
pTag

<p class="fbs-auth__title">Sign in to your Forbes account <span class="light-gray">or</span></p>

In [118]:
soup.contents[1].contents[1].contents[2].contents[-3].contents[-1].contents[-2]

<div class="article-wrapper"><div article-index="0" class="left-rail"></div><div class="body-container"><div class="article-headline-container"><div class="header-content-container"><div class="metrics-channel light-text"><div class="content-data light-text"><span class="hidden pageviews-wrapper"></span><span class="hidden vert-pipe">|</span><time>May 2, 2020,</time><span class="time"><time>02:38pm EDT</time></span></div></div><h1 class="fs-headline speakable-headline font-base font-size">WhatsApp Suddenly Gets Powerful New Security Boost: Here’s Why It Affects You</h1></div></div><div class="top-contrib-block"><div class="contribs"><div class="contrib-container top-contrib"><a class="fs-author-avatar" href="https://www.forbes.com/sites/zakdoffman/" title="Photo of Zak Doffman"><img alt="Zak Doffman" class="fs-author-image" src="https://secure.gravatar.com/avatar/9e81c73adca937778db3039d8ba72b45?s=400&amp;d=mm&amp;r=g"/></a><div class="contrib-byline"><div class="fs-author-wrapper"><sp

In [51]:
from newsfetch.news import newspaper
news = newspaper(url)

In [63]:
url

'https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/'

In [52]:
dir(news)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'article',
 'authors',
 'category',
 'date_download',
 'date_modify',
 'date_publish',
 'description',
 'filename',
 'get_dict',
 'headline',
 'image_url',
 'keywords',
 'publication',
 'source_domain',
 'summary',
 'title_page',
 'title_rss',
 'uri']

In [62]:
news.get_dict

{'headline': '',
 'author': ['Zak Doffman'],
 'date_publish': '2020-05-02 00:00:00',
 'date_modify': 'None',
 'date_download': '2020-07-01 16:30:44',
 'image_url': 'en',
 'filename': 'https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/.json',
 'description': None,
 'publication': None,
 'category': None,
 'source_domain': 'www.forbes.com',
 'article': '',
 'summary': '',
 'keyword': [],
 'title_page': None,
 'title_rss': None,
 'url': 'https://www.forbes.com/sites/zakdoffman/2020/05/02/whatsapp-suddenly-gets-two-powerful-security-surprises-heres-why-this-matters/'}

In [123]:
test = newspaper('https://www.bbc.com/news/world-48810070')

In [126]:
test.article

'Media playback is unsupported on your device Media caption Trump on China, including a change of US policy on the tech giant Huawei The US and China have agreed to resume trade talks, easing a long row that has contributed to a global economic slowdown. US President Donald Trump and China\'s President Xi Jinping reached agreement at the G20 summit in Japan. Mr Trump also said he would allow US companies to continue to sell to the Chinese tech giant Huawei, in a move seen as a significant concession. Mr Trump had threatened additional trade sanctions on China. However, after the meeting on the sidelines of the main G20 summit in Osaka, he confirmed that the US would not be adding tariffs on $300bn (PS236bn) worth of Chinese imports. He also said he would continue to negotiate with Beijing "for the time being". And at a subsequent press conference, the US president declared that US technology companies could again sell to China\'s Huawei - effectively reversing a ban imposed last month 

In [129]:
QQQdf['url'][19]

'https://cincodias.elpais.com/cincodias/2020/05/01/companias/1588360448_615537.html'

In [ ]:
test = newspaper(QQQdf['url'][19])


In [130]:
test.article

'Media playback is unsupported on your device Media caption Trump on China, including a change of US policy on the tech giant Huawei The US and China have agreed to resume trade talks, easing a long row that has contributed to a global economic slowdown. US President Donald Trump and China\'s President Xi Jinping reached agreement at the G20 summit in Japan. Mr Trump also said he would allow US companies to continue to sell to the Chinese tech giant Huawei, in a move seen as a significant concession. Mr Trump had threatened additional trade sanctions on China. However, after the meeting on the sidelines of the main G20 summit in Osaka, he confirmed that the US would not be adding tariffs on $300bn (PS236bn) worth of Chinese imports. He also said he would continue to negotiate with Beijing "for the time being". And at a subsequent press conference, the US president declared that US technology companies could again sell to China\'s Huawei - effectively reversing a ban imposed last month 

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

# method 7.2

use api from various companies on rapid api

yahoo finance news api

seems only latest news

In [2]:
import requests

url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/ne/news/AAPL"

headers = {
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
    'x-rapidapi-key': "sign up to get the key"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{
    "item": [
        {
            "description": "Yahoo Finance\u2019s Dan Howley joins Sibile Marcellus to discuss how CEOs from Apple, Google, Amazon and Facebook will reportedly testify before Congress to answer questions related to its antitrust investigation of Big Tech. ",
            "guid": "a6ca9610-5e62-3e94-bfb7-418086e3af7d",
            "link": "https://finance.yahoo.com/video/big-tech-ceos-testify-congress-192004023.html?.tsrc=rss",
            "pubDate": "Thu, 02 Jul 2020 19:20:04 +0000",
            "title": "Big tech CEOs will testify before Congress on antitrust inquiry: RPT"
        },
        {
            "description": "Looking at the stock market today, the first thought that comes to mind is that it is divorced from economic reality Continue reading...",
            "guid": "b0b9d598-407d-34f8-afff-f1d497845045",
            "link": "https://finance.yahoo.com/news/nifty-faang-other-one-decision-174204653.html?.tsrc=rss",
            "pubDate": "Thu, 02 Jul 2

In [5]:
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [14]:
response.json()['item']

[{'description': 'Yahoo Finance’s Dan Howley joins Sibile Marcellus to discuss how CEOs from Apple, Google, Amazon and Facebook will reportedly testify before Congress to answer questions related to its antitrust investigation of Big Tech. ',
  'guid': 'a6ca9610-5e62-3e94-bfb7-418086e3af7d',
  'link': 'https://finance.yahoo.com/video/big-tech-ceos-testify-congress-192004023.html?.tsrc=rss',
  'pubDate': 'Thu, 02 Jul 2020 19:20:04 +0000',
  'title': 'Big tech CEOs will testify before Congress on antitrust inquiry: RPT'},
 {'description': 'Looking at the stock market today, the first thought that comes to mind is that it is divorced from economic reality Continue reading...',
  'guid': 'b0b9d598-407d-34f8-afff-f1d497845045',
  'link': 'https://finance.yahoo.com/news/nifty-faang-other-one-decision-174204653.html?.tsrc=rss',
  'pubDate': 'Thu, 02 Jul 2020 17:42:04 +0000',
  'title': 'Nifty FAANG and Other &#39;One Decision&#39; Investment Strategies'},
 {'description': "The Dow's rally dro

In [16]:
import pandas as pd

df = pd.DataFrame(response.json()['item'])

In [17]:
df.shape

(18, 5)

In [18]:
df

,description,guid,link,pubDate,title
0,Yahoo Finance’s Dan Howley joins Sibile Marcel...,a6ca9610-5e62-3e94-bfb7-418086e3af7d,https://finance.yahoo.com/video/big-tech-ceos-...,"Thu, 02 Jul 2020 19:20:04 +0000",Big tech CEOs will testify before Congress on ...
1,"Looking at the stock market today, the first t...",b0b9d598-407d-34f8-afff-f1d497845045,https://finance.yahoo.com/news/nifty-faang-oth...,"Thu, 02 Jul 2020 17:42:04 +0000",Nifty FAANG and Other &#39;One Decision&#39; I...
2,The Dow's rally drove shares of Apple (NASDAQ:...,0b39231d-89cb-3974-a6f9-e3122339100c,https://finance.yahoo.com/m/0b39231d-89cb-3974...,"Thu, 02 Jul 2020 17:31:00 +0000",Dow Jones Rises on Jobs Report; Apple and McDo...
3,Big Tech&#39;s four biggest executives may soo...,1169ab0b-9cff-3119-b1db-2df784b7d106,https://finance.yahoo.com/news/ceos-facebook-a...,"Thu, 02 Jul 2020 17:10:18 +0000","CEOs of Facebook, Amazon, Apple and Google to ..."
4,The U.S. State Department warned top American ...,d1f70957-f3b1-3cd7-8072-02f5f142a7ab,https://finance.yahoo.com/news/state-dept-warn...,"Thu, 02 Jul 2020 16:55:06 +0000",REFILE-State Dept warns top U.S. firms over su...
5,In the face of the sudden and sharp increases ...,01d66751-ec4c-3c1c-85ba-248d25ef7cfd,https://finance.yahoo.com/m/01d66751-ec4c-3c1c...,"Thu, 02 Jul 2020 15:39:00 +0000",Apple Re-Closing 77 Stores Due to the Coronavirus
6,"Roku (ROKU) adds virtual fitness platform, Pel...",85b83273-e0d9-352c-adaf-35c0a1cc4fa6,https://finance.yahoo.com/news/roku-roku-expan...,"Thu, 02 Jul 2020 14:20:02 +0000",Roku (ROKU) Expands Health &amp; Fitness Optio...
7,Shares of Apple (NASDAQ: AAPL) rose 14.7% in J...,cfbc8583-f39f-3f71-93ea-2968a2a08811,https://finance.yahoo.com/m/cfbc8583-f39f-3f71...,"Thu, 02 Jul 2020 14:06:00 +0000",Why Apple&#39;s Shares Rose 14.7% in June
8,"Apple, Google, Amazon and Facebook CEOs have a...",8e22d1bc-8af6-3eb0-a0a2-5e8acecfddee,https://finance.yahoo.com/video/tech-leaders-a...,"Thu, 02 Jul 2020 14:05:11 +0000",Tech leaders agree to testify on Capitol Hill
9,(Bloomberg Opinion) -- If you’re concerned abo...,c72dd876-9d12-31de-99c9-18fff61f0ea9,https://finance.yahoo.com/news/really-want-goo...,"Thu, 02 Jul 2020 13:20:11 +0000",Do You Really Want Google to Have Your Persona...


# method 7.4

diana news and analysis

it may provide better data quality about news as it focus on only news data

In [19]:
import requests

url = "https://diana-news-and-analysis.p.rapidapi.com/dates/201901/keyword"

querystring = {"Date":"201902","Search-Term":"AAPL"}

headers = {
    'x-rapidapi-host': "diana-news-and-analysis.p.rapidapi.com",
    'x-rapidapi-key': "sign up to get the key"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"message": "Network error communicating with endpoint"}


In [20]:
import requests

url = "https://diana-news-and-analysis.p.rapidapi.com/articles/20150101/AAPL"

headers = {
    'x-rapidapi-host': "diana-news-and-analysis.p.rapidapi.com",
    'x-rapidapi-key': "6e4f46d1bdmsh66a472641a03727p167d6ajsn1bee51013ed4"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

{"message": "Network error communicating with endpoint"}


In [ ]:
df = pd.DataFrame(response.json()['item'])

# method 7.5

bloomberg

seems no news data

In [21]:
import requests

url = "https://bloomberg-market-and-financial-news.p.rapidapi.com/stock/get-financials"

querystring = {"id":"aapl%3Aus"}

headers = {
    'x-rapidapi-host': "bloomberg-market-and-financial-news.p.rapidapi.com",
    'x-rapidapi-key': "sign up to get the key"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"result":[{"name":"Income Statement","securityID":"aapl:us","timeBasedSheets":[{"name":"Quarterly","columnHeadings":["9/2019","12/2019","3/2020"],"chartData":[{"name":"Revenue","chartType":"bar","isPercentage":false,"values":[6.404E10,9.1819E10,5.8313E10]},{"name":"Net Income","chartType":"bar","isPercentage":false,"values":[1.3686E10,2.2236E10,1.1249E10]},{"name":"Profit Margin","chartType":"line","isPercentage":true,"values":[21.371,24.2172,19.2907]}]},{"name":"Annual","columnHeadings":["2016","2017","2018","2019"],"chartData":[{"name":"Revenue","chartType":"bar","isPercentage":false,"values":[2.15639E11,2.29234E11,2.65595E11,2.60174E11]},{"name":"Net Income","chartType":"bar","isPercentage":false,"values":[4.5687E10,4.8351E10,5.9531E10,5.5256E10]},{"name":"Profit Margin","chartType":"line","isPercentage":true,"values":[21.1868,21.0924,22.4142,21.2381]}]}]},{"name":"Balance Sheet","securityID":"aapl:us","timeBasedSheets":[{"name":"Quarterly","columnHeadings":["9/2019","12/2019","3/2

In [23]:
len(response.json()['result'])

3

In [26]:
response.json()['result'][0]

{'name': 'Income Statement',
 'securityID': 'aapl:us',
 'timeBasedSheets': [{'name': 'Quarterly',
   'columnHeadings': ['9/2019', '12/2019', '3/2020'],
   'chartData': [{'name': 'Revenue',
     'chartType': 'bar',
     'isPercentage': False,
     'values': [64040000000.0, 91819000000.0, 58313000000.0]},
    {'name': 'Net Income',
     'chartType': 'bar',
     'isPercentage': False,
     'values': [13686000000.0, 22236000000.0, 11249000000.0]},
    {'name': 'Profit Margin',
     'chartType': 'line',
     'isPercentage': True,
     'values': [21.371, 24.2172, 19.2907]}]},
  {'name': 'Annual',
   'columnHeadings': ['2016', '2017', '2018', '2019'],
   'chartData': [{'name': 'Revenue',
     'chartType': 'bar',
     'isPercentage': False,
     'values': [215639000000.0,
      229234000000.0,
      265595000000.0,
      260174000000.0]},
    {'name': 'Net Income',
     'chartType': 'bar',
     'isPercentage': False,
     'values': [45687000000.0, 48351000000.0, 59531000000.0, 55256000000.0]},

# method 7.6

tradesly

In [29]:
import requests

url = "https://tradeslyapi.p.rapidapi.com/stock-news/search"

payload = "{  \"symbol\": \"TSLA\"}"
headers = {
    'x-rapidapi-host': "tradeslyapi.p.rapidapi.com",
    'x-rapidapi-key': "6e4f46d1bdmsh66a472641a03727p167d6ajsn1bee51013ed4",
    'content-type': "application/json",
    'accept': "application/json"
    }

response = requests.request("POST", url, data=payload, headers=headers)

print(response.text)

[{"date":"Thu, 02 Jul 2020 17:24:00 -0400","image_url":"https://cdn.snapi.dev/images/v1/a/g/2019-tesla-model-s-long-range-44.jpg","news_url":"https://www.fool.com/investing/2020/07/02/nasdaq-hits-new-records-as-tesla-delivers-keeps-so.aspx","sentiment":"Positive","source_name":"The Motley Fool","text":"Find out why the electric automaker has been such a big part of the Nasdaq's success.","tickers":["TSLA"],"title":"Nasdaq Hits New Records as Tesla Delivers, Keeps Soaring","topics":[],"type":"Article"},{"date":"Thu, 02 Jul 2020 16:34:58 -0400","image_url":"https://cdn.snapi.dev/images/v1/x/c/tesla-could-hit-2000-a-share-wedbushs-dan-ives.jpg","news_url":"https://www.youtube.com/watch?v=5VfFFouFTBY","sentiment":"Positive","source_name":"Yahoo Finance","text":"On Thursday, Wedbush analyst Dan Ives raised his bull case price target for shares of Tesla to $2,000 from $1,500.","tickers":["TSLA"],"title":"Tesla could hit $2000 a share: Wedbush's Dan Ives","topics":[],"type":"Video"},{"date":"

In [31]:
len(response.json() )

20

In [32]:
df = pd.DataFrame(response.json())

In [33]:
df.shape

(20, 10)

In [35]:
df.tail()

,date,image_url,news_url,sentiment,source_name,text,tickers,title,topics,type
15,"Thu, 02 Jul 2020 10:53:52 -0400",https://cdn.snapi.dev/images/v1/p/f/105780169-...,https://247wallst.com/autos/2020/07/02/why-tes...,Positive,24/7 Wall Street,"Over the past 12 months, the Nasdaq Composite ...",[TSLA],Why Tesla Is the World’s Most Dominant Automaker,[],Article
16,"Thu, 02 Jul 2020 10:42:47 -0400",https://cdn.snapi.dev/images/v1/w/e/wew22wesss...,https://www.forbes.com/sites/alanohnsman/2020/...,Positive,Forbes,Quarterly deliveries topped analyst expectatio...,[TSLA],Tesla’s Electric Car Deliveries Sail Past Expe...,[product],Article
17,"Thu, 02 Jul 2020 10:25:13 -0400",https://cdn.snapi.dev/images/v1/s/r/urlhttps3a...,https://nypost.com/2020/07/02/tesla-crushes-co...,Positive,New York Post,"Shares of Elon Musk’s electric car company, wh...",[TSLA],Tesla crushes coronavirus delivery expectation...,[earnings],Article
18,"Thu, 02 Jul 2020 10:23:00 -0400",https://cdn.snapi.dev/images/v1/u/4/shuttersto...,https://www.zacks.com/stock/news/996969/unempl...,Negative,Zacks Investment Research,Unemployment Rate Decline to 11.1%,[TSLA],Unemployment Rate Decline to 11.1%,[],Article
19,"Thu, 02 Jul 2020 10:21:00 -0400",https://cdn.snapi.dev/images/v1/o/r/105520081-...,https://www.fool.com/investing/2020/07/02/why-...,Positive,The Motley Fool,The stock is hitting new highs following bette...,[TSLA],Why Tesla Stock Soared Higher on Thursday,[],Article


# use method 7

In [165]:
import requests

#use following process to extract company info from finnhub.io
#A. company = “SK.PA” / api_key = keys [‘finnhub’]
#B. url = “https://finnhub.io/api/v1/stock/profile?symbol={}&token={}".format(company, api_key)
#C. req = requests.get(url)

company = 'AAPL'
start_date = '2020-01-01'
end_date = '2020-01-26'
api_key = 'brudnjvrh5r8570qt1r0'

api_company_news = 'https://finnhub.io/api/v1/company-news?symbol={}&from={}&to={}&token={}'.format(company, start_date, end_date, api_key)
print('visit ',api_company_news)
r = requests.get(api_company_news)
#print(r.json())

visit  https://finnhub.io/api/v1/company-news?symbol=AAPL&from=2020-01-01&to=2020-01-26&token=brudnjvrh5r8570qt1r0


In [166]:
#assume keys in each json data are the same
df = pd.DataFrame(r.json())

In [167]:
#r.json()

In [168]:
df.shape

(200, 9)